Проект №3 EDA Booking.com
=

## Какой кейс решаем?

Представим, что работаем дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели, которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно, отель ведёт себя нечестно, и его стоит проверить.

## Наименование столбцов:
- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве.
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.


## Импортируем все необходимое

In [ ]:
!pip install Afinn
!pip install nltk
# импортируем библиотеки
import numpy as np
import pandas as pd
import category_encoders as ce

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_rows', 200)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import re
import string
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from afinn import Afinn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
RANDOM_SEED = 42

# Исследование данных

In [ ]:
DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR + 'hotels_train.csv')
df_test = pd.read_csv(DATA_DIR + 'hotels_test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'submission.csv')

In [ ]:
df_train.info()

In [ ]:
dupl_columns = list(df_train.columns)

mask_d = df_train.duplicated(subset=dupl_columns)
df_train_duplicates = df_train[mask_d]

df_train = df_train.drop_duplicates(subset=dupl_columns)

df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_test.info()

In [ ]:
df_test.head()

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.head()

Для обработки признаков обьединим тренировочный и тестовый датасет в один, для разделения добавим общий признак sample, который будет равен 1 для тренировочных данных и 0 для тестовых. Так же в тестовом датасете создадим целевой признак reviewer_score и заполним его нулями.

In [ ]:
df_train['sample'] = 1
df_test['sample'] = 0
df_test['reviewer_score'] = 0

hotels = pd.concat([df_train, df_test], sort=False).reset_index(drop=True)

In [ ]:
hotels.info()

In [ ]:
hotels.head()

In [ ]:
hotels.nunique(dropna=False)

# Проектирование признаков

## Заполнение пропущенных значений

Выделим признаки с пропусками в данных.

In [ ]:
hotels_nulls = hotels.isnull().sum()
display(hotels_nulls[hotels_nulls > 0])

В признаках lat и lng есть пропуски,но их очень мало, значит надо заполнить пропущенные значения.

In [ ]:
def fill_missing_with_mean(df, columns):
    for column in columns:
        mean_value = df[column].mean()
        df[column] = df[column].fillna(mean_value)

# Заполнение пропущенных значений в столбцах 'lat' и 'lng'
columns_to_fill = ['lat', 'lng']
fill_missing_with_mean(hotels, columns_to_fill)

# Визуализация данных

In [ ]:
# Распределение оценок рецензентов
plt.figure(figsize=(10, 6))
sns.histplot(hotels['reviewer_score'], bins=20, kde=True)
plt.title('Распределение оценок рецензентов')
plt.xlabel('Оценка рецензента')
plt.ylabel('Частота')
plt.show()

Вывод:
Распределение оценок рецензентов показывает, что большинство отзывов имеют высокие оценки. Есть пик в районе низких оценок, что указывает на наличие некоторого числа негативных отзывов. В целом, большинство рецензентов довольны своим пребыванием в отелях

In [ ]:
# Распределение отзывов по времени
plt.figure(figsize=(12, 6))
hotels['review_date'].value_counts().sort_index().plot()
plt.title('Распределение отзывов во времени')
plt.xlabel('Дата')
plt.ylabel('Количество отзывов')
plt.show()

Вывод:
Линейный график показывает количество отзывов по времени. Можно заметить сезонные колебания, что указывает на пики в туристический сезон. В последние годы можно заметить тенденцию к увеличению числа отзывов, что может свидетельствовать о росте популярности онлайн-отзывов.

In [ ]:
# Частота тегов
tags_flat = [tag for sublist in hotels['tags'].apply(lambda x: x.strip("[]").split(", ")) for tag in sublist]
tags_flat = [tag.strip("' ") for tag in tags_flat]
tags_series = pd.Series(tags_flat)

plt.figure(figsize=(12, 6))
sns.barplot(x=tags_series.value_counts().index[:20], y=tags_series.value_counts().values[:20])
plt.xticks(rotation=90)
plt.title('Топ 20 самых частых тегов')
plt.xlabel('Теги')
plt.ylabel('Частота')
plt.show()

Вывод:
Столбчатая диаграмма показывает наиболее часто встречающиеся теги. Теги, связанные с типом поездки (например, "Leisure trip", "Couple", "Business trip"), являются наиболее распространенными. Это дает представление о типе гостей, посещающих отели, и их мотивациях.

## Общие выводы:
- Большинство гостей дают высокие оценки отелям, что говорит о высоком уровне удовлетворенности.
- Однодневные поездки наиболее распространены среди гостей, что может указывать на большую долю бизнес-путешественников.
- Тенденция к увеличению числа отзывов указывает на рост популярности онлайн-платформ для отзывов.
- Часто используемые теги показывают, что большая часть гостей приезжает на отдых или в деловые поездки.

# Проектирование признаков

## Название отеля

In [ ]:
# получаем количество уникальных значений отеля по названию
hotels['hotel_name'].nunique()

In [ ]:
# Выделим отели под одинаковым названием, но с различным адресом и добавим, для уникальности, к названию часть адреса.
address_list = hotels.pivot_table(
    values = 'hotel_address',
    index = 'hotel_name',
    aggfunc = 'nunique'
)
dupl_hotel = address_list[address_list['hotel_address'] > 1]
display(dupl_hotel)
_temp_list = dupl_hotel[dupl_hotel['hotel_address'] > 1].index.to_list()
hotels['hotel_name'] = hotels.apply(lambda x: (x['hotel_name'] + ' in ' + x['hotel_address'].split()[-1])
    if (x['hotel_name'] in _temp_list) else x['hotel_name'], axis=1)
display(hotels['hotel_name'].nunique())

Окончательно получили 1494 уникальных отеля.

## Адрес отеля

In [ ]:
 hotels['hotel_address'].head(20)

Из признака hotel_address извлекаем страну и город.

In [ ]:
hotels['hotel_country'] = hotels['hotel_address'].apply(lambda x: x.split()[-1])
hotels['hotel_country'] = hotels['hotel_country'].apply(lambda x:'United Kingdom' if x == 'Kingdom' else x)
country_list = list(hotels['hotel_country'].unique())
display(country_list)

In [ ]:
hotels['hotel_city'] = hotels['hotel_address'].apply(lambda x: x.split()[-2])
hotels['hotel_city'] = hotels['hotel_city'].apply(lambda x: 'London' if x == 'United' else x)
city_list = list(hotels['hotel_city'].unique())
display(city_list)

In [ ]:
hotels['hotel_country_code'] = hotels['hotel_country'].astype('category').cat.codes
hotels.head()

Получем 6 унакльных стран и 6 уникальных городов.

## Географические координаты

Несмотря на то, что данные представлены в числовом виде правильнее их рассматривать как категориальные. считаю что стоит избавиться от этих данных

In [ ]:
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x.strip())

In [ ]:
hotels = hotels.drop(['lat', 'lng'], axis=1)
hotels.head()

## Национальность рецензента.

In [ ]:
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x.strip())

Сделаем признак того, что рецензент является гражданином страны, в котором расположен отель

In [ ]:
hotels['is_citizen'] = hotels['reviewer_nationality'] == hotels['hotel_country']
hotels['is_citizen'] = hotels['is_citizen'].astype('int')

В наборе данных 227 уникальных значений для признака национальности рецензента. Закодируем топ 10, остальных отметим как other.

In [ ]:
top_nations = list(hotels['reviewer_nationality'].value_counts()[:10].index)
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x if x in top_nations else 'other')
hotels['reviewer_nationality_code'] = hotels['reviewer_nationality'].astype('category').cat.codes

Европейские страны по большей части христиане, выделим возможных рецензентов-мусульман.

In [ ]:
muslim_countries = ['Turkey', 'United Arab Emirates', 'Saudi Arabia', 'Qatar', 'Kuwait', 'Oman']
hotels['possible_muslim'] = hotels['reviewer_nationality'].apply(lambda x: 1 if x in muslim_countries else 0)

In [ ]:
hotels.head()

## Тэги

In [ ]:
# количество тегов
hotels['tags_count'] = hotels['tags'].apply(lambda x: len(re.findall("'[^\'](.+?)[^\']'", x)))

Создадим новые признаки на основании тегов.

In [ ]:
def get_tags(tags_string):
    # Удаляем ненужные символы
    tags_string = tags_string.strip("[]").replace("' ", '').replace(" '", '')
    tags_list = [tag.strip() for tag in tags_string.split(',')]

    reviewer_types = {
        'Solo traveler': 1,
        'Couple': 2,
        'Travelers with friends': 3,
        'Family with young children': 4,
        'Family with older children': 5,
        'Group': 6
    }

    room_types = {
        'without Window': 0,
        'Guestroom': 1,
        'Classic': 2,
        'Single': 3,
        'Standard': 4,
        'Superior': 5,
        'Comfort': 6,
        'Club': 7,
        'Suite': 8,
        'Deluxe': 9,
        'King': 10,
        'Premier': 11
    }

    tag_from_mobile = 0
    tag_with_pet = 0
    tag_leisure_trip = 1
    tag_reviewer_type = 0
    tag_count_nights = 0
    tag_view_room = 0
    tag_room_type = -1

    for tag in tags_list:
        if tag == 'Submitted from a mobile device':
            tag_from_mobile = 1
        elif tag == 'With a pet':
            tag_with_pet = 1
        elif tag == 'Business trip':
            tag_leisure_trip = 0
        elif tag == 'Leisure trip':
            tag_leisure_trip = 1
        elif tag in reviewer_types:
            tag_reviewer_type = reviewer_types[tag]
        elif re.fullmatch(r'Stayed\s\d+\snight\w?', tag):
            tag_count_nights = int(re.findall(r'\d+', tag)[0])
        elif 'View' in tag or 'Panoramic' in tag:
            tag_view_room = 1
        elif tag in room_types:
            tag_room_type = room_types[tag]
        elif 'Double' in tag or 'Twin' in tag:
            tag_room_type = 12

    if tag_room_type == -1:
        tag_room_type = 13

    return pd.Series([tag_from_mobile, tag_with_pet, tag_leisure_trip,
                      tag_reviewer_type, tag_count_nights, tag_view_room,
                      tag_room_type])

# Применяем функцию и создаем новые столбцы
hotels[['tag_from_mobile', 'tag_with_pet', 'tag_leisure_trip',
        'tag_reviewer_type', 'tag_count_nights', 'tag_view_room',
        'tag_room_type']] = hotels['tags'].apply(get_tags)

In [ ]:
hotels.head()

## Дата отзыва

In [ ]:
# Выделим из даты отзыва месяц.
hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['review_month'] = hotels['review_date'].dt.month

In [ ]:
# Выделим день недели
hotels['day_of_week'] = hotels['review_date'].dt.dayofweek

In [ ]:
# Выделим месяцы "высокого" сезона.
display(hotels['review_month'].value_counts())

In [ ]:
# Выделим топ 4
hotels['high_season'] = hotels['review_month'].apply(lambda x: 1 if x in [8,7,5,10] else 0)

In [ ]:
# Заменим признак days_since_review на числовой.
hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: np.float64(re.findall(r'\d+', x)[0])).convert_dtypes()

In [ ]:
# Общее количество отзывов
hotels['total_number_of_reviews_reviewer_has_given'] = hotels['total_number_of_reviews_reviewer_has_given'].apply(lambda x: np.sqrt(x))

In [ ]:
hotels.head()

## Отзывы

Отзывы и их структура поможет разобраться в поставленной задаче лучше всего. Для этого надо изучить отзывы, выделить эмоциональную окраску: негативная, нейтральная, позитивная.

In [ ]:
# Создаём объекты SentimentIntensityAnalyzer
positive_analyzer = SentimentIntensityAnalyzer()
negative_analyzer = SentimentIntensityAnalyzer()

# Посмотрим, как изменится характеристика слова 'nothing'
example_word = 'nothing'
print(f'Словарь оценок слова "{example_word}" до преобразований:')
print(negative_analyzer.polarity_scores(example_word), '\n')

# Добавим контекст
negative_words = {'small': -1, 'nothing': 10}
positive_words = {'nothing': -10, 'everything': 10}

negative_analyzer.lexicon.update(negative_words)
positive_analyzer.lexicon.update(positive_words)

print(f' - после преобразований (для негативного контекста):')
print(negative_analyzer.polarity_scores(example_word))
print(f' - после преобразований (для позитивного контекста):')
print(positive_analyzer.polarity_scores(example_word))

# Пример с фразами "No Negative" и "Positive"
phrases = ['No Negative', 'Positive']
for phrase in phrases:
    print(f'\nСловарь оценок фразы "{phrase}":')
    print(negative_analyzer.polarity_scores(phrase))

# Обновление меток отсутствия частей отзыва
# Определяем словарь замен
replacements = {
    'No Negative': 'Positive',
    'N A': 'Positive',
    'All good': 'Positive',
    'No complaints': 'Positive',
    'Nothing to dislike': 'Positive',
    'No Positive': 'Negative'
}

# Функция для замены значений
def replace_phrases(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

# Применяем функцию к столбцам
hotels['negative_review_temp'] = hotels['negative_review'].apply(lambda x: replace_phrases(x, replacements))
hotels['positive_review_temp'] = hotels['positive_review'].apply(lambda x: replace_phrases(x, replacements))

In [ ]:
%%time

# Функция для извлечения оценок
def get_sentiments(df, text_column, analyzer, prefix):
    sentiments = df[text_column].apply(analyzer.polarity_scores).apply(pd.Series)
    sentiments.columns = [f'{prefix}_{col}' for col in sentiments.columns]
    return sentiments

# Получаем оценки для негативных отзывов
negative_sentiments = get_sentiments(hotels, 'negative_review_temp', negative_analyzer, 'neg_review_sentiments')
# Получаем оценки для позитивных отзывов
positive_sentiments = get_sentiments(hotels, 'positive_review_temp', positive_analyzer, 'pos_review_sentiments')

# Объединяем результаты с исходным DataFrame
hotels = pd.concat([hotels, negative_sentiments, positive_sentiments], axis=1)

# Отбор призаков

## Очистка дата-сета от ненужных вспомогательных признаков

In [ ]:
hotels.info()

In [ ]:
hotels_copy = hotels.copy()

In [ ]:
object_columns = [s for s in hotels_copy.columns if hotels_copy[s].dtypes == 'object']
hotels_copy.drop(object_columns, axis = 1, inplace=True)

In [ ]:
hotels_copy.info()

In [ ]:
cat_date_columns = [cd for cd in hotels_copy.columns if ((hotels_copy[cd].dtypes.name == 'category') or (hotels_copy[cd].dtypes == 'datetime64[ns]'))]
hotels_copy.drop(cat_date_columns, axis = 1, inplace=True)

In [ ]:
hotels_copy.info()

## Анализ мультиколлинеарности

In [ ]:
cols_num = [
    'review_total_negative_word_counts',
    'review_total_positive_word_counts',
    'total_number_of_reviews_reviewer_has_given',
    'tags_count',
    'tag_count_nights',
    'neg_review_sentiments_neg',
    'neg_review_sentiments_neu',
    'neg_review_sentiments_pos',
    'neg_review_sentiments_compound',
    'pos_review_sentiments_neg',
    'pos_review_sentiments_neu',
    'pos_review_sentiments_pos',
    'pos_review_sentiments_compound'
]

cols_cat = [
    'additional_number_of_scoring',
    'average_score',
    'total_number_of_reviews',
    'days_since_review',
    'hotel_country_code',
    'reviewer_nationality_code',
    'is_citizen',
    'possible_muslim',
    'reviewer_score',
    'is_citizen',
    'possible_muslim',
    'tag_from_mobile',
    'tag_with_pet',
    'tag_leisure_trip',
    'tag_reviewer_type',
    'tag_view_room',
    'tag_room_type',
    'review_month',
    'day_of_week',
    'high_season',
    'review_month',
]

In [ ]:
def show_corr_heatmap(columns_list, title, method):
    fig_, ax_ = plt.subplots(figsize=(15, 12))
    corr = hotels_copy[columns_list].corr(method=method)
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr,
                annot=True,
                linewidths=0.1,
                ax=ax_,
                mask=mask,
                cmap='viridis',
                fmt='.1g')
    ax_.set_title(title, fontsize=18)
    plt.show()

In [ ]:
show_corr_heatmap(cols_num, 'корреляция непрерывных признаков', method='pearson')

In [ ]:
drop_columns = ['neg_review_sentiments_pos']

In [ ]:
show_corr_heatmap(cols_cat, 'корреляция непрерывных признаков', method='spearman')

In [ ]:
drop_columns += ['review_month', 'is_citizen', 'possible_muslim', 'additional_number_of_scoring']

In [ ]:
hotels_copy.drop(drop_columns, axis = 1, inplace=True)

In [ ]:
hotels_copy.info()

## Оценка значимости признаков

Для оценки значимости категориальных признаков будем использовать тест хи-квадрат, для непрерывных признаков — тест ANOVA.

In [ ]:
y = hotels_copy.query('sample == 1').drop(['sample'], axis=1)['reviewer_score'].values
X = hotels_copy.query('sample == 1').drop(['sample', 'reviewer_score'], axis=1)

# визуализируем результат анализа значимости:
imp_num = pd.Series(f_classif(X[X.columns], y)[0], index = X.columns)
imp_num.sort_values(inplace = True)

fig5, ax5 = plt.subplots(figsize=(15, 20))
imp_num.plot(kind = 'barh', color='red')

In [ ]:
imp_num

In [ ]:
drop_columns = imp_num[imp_num < 5].index.tolist()
drop_columns

In [ ]:
hotels_copy.drop(drop_columns, axis = 1, inplace=True)

# Обучение модели и получение предсказания

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
train_data = hotels_copy.query('sample == 1').drop(['sample'], axis=1)
test_data = hotels_copy.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values
X = train_data.drop(['reviewer_score'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
model = RandomForestRegressor(
    n_estimators=100,
    verbose=1,
    n_jobs=-1,
    random_state=RANDOM_SEED)

In [ ]:
%%time

# обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
def mean_absolute_percentage_error(y_tr, y_pr):
    y_tr, y_pr = np.array(y_tr), np.array(y_pr)
    return np.mean(np.abs((y_tr - y_pr) / y_tr)) * 100

print('MAPE:', round(mean_absolute_percentage_error(y_test, y_pred), 2))

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh', color='green');

Получаем предсказание целевой переменной

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)
predict_submission = model.predict(test_data)
sample_submission['reviewer_score'] = predict_submission

#  сохраняем результат:
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)